In [1]:
import csv

from pathlib import Path

from owlready2 import *

import pandas as pd

from onto_classes import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# set up out path
data_p = Path("../../data/")

# set paths
ontology_p = "../../data/"

# set onto path
upper_p   = ontology_p + "/nourish-merged-2.owl"

In [3]:
# load ontologies
onto = get_ontology(upper_p).load()

In [4]:
for c in onto.classes():
    # if c.name == "IAO_0000078": # curation status -> OneOf example
    # if c.name == "FOODON_00002396":
    if c.name == "FOODON_00002403": #food_material
    # if c.name == "FOODON_00002114":
    # if c.name == "CDNO_0000001": # breaks cause no label
        break

In [5]:
print(f"label: {c.label}\nEquivalent to: {c.equivalent_to[0]}")

label: ['food material']
Equivalent to: obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)


In [6]:
# test making graph
entity_set = set()
transition_set = set()

# Add node and create graph
root_node = EntityNode(c, "Concept")
g = Graph(root_node, entity_set, transition_set)

# Add children to root node
if list(c.equivalent_to):
    for child in c.equivalent_to:
        parse_logic(child, root_node,
                    edge="equivalent_to", edge_direction="target",
                    restr_key=None, restr_value=None
                    )

elif list(c.is_a):
    for child in c.is_a:
        parse_logic(child, root_node,
                    edge="is_a", edge_direction="target",
                    restr_key=None, restr_value=None
                    )

else:
    print(f"{c} does not have equivalent_to or is_a properties.")

In [7]:
g.root.children

[{'node': <onto_classes.TransitionNode at 0x116191570>,
  'direction': 'target',
  'rel_type': 'equivalent_to',
  'rel_label': '',
  'restriction': None,
  'restriction_value': None}]

In [8]:
root_node.children[0]["node"].children

[{'node': <onto_classes.EntityNode at 0x1161926b0>,
  'direction': 'source',
  'rel_type': 'member_of',
  'rel_label': '',
  'restriction': None,
  'restriction_value': None},
 {'node': <onto_classes.TransitionNode at 0x1088514b0>,
  'direction': 'source',
  'rel_type': 'member_of',
  'rel_label': '',
  'restriction': None,
  'restriction_value': None}]

In [9]:
def export_traverse(node, parent_id=None):
    match node:
        case Graph():
            # initialize recursion
            # node_children = node.root.children
            
            print(node.root.id)
            print(create_node_row(node.root))
            node = node.root
            print('------------')
            
            
            # write out edges and parent node
            # TODO
        case EntityNode():
             # node_children = node.children
            pass
        case TransitionNode():
            # node_children = node.children
            # if transitionNode has no children -> return and do not write empty node
            if len(node.children) == 0:
                return
            
            
    for n in node.children:
        print(n['node'].id)
        print(create_node_row(n['node']))
        print(create_edge_row(node, n))
        print('vvvvvvvvvvvvv')
        export_traverse(n['node'])
        
        # write out edges
        #TODO
            
def create_node_row(n):
    # node_id:ID,label:string[],iri,:LABEL
    return [n.id, n.label, n.iri, n.neo_type]

def create_edge_row(parent_n, child_n):
    # :START_ID,:END_ID,:TYPE,restriction,restriction_value,label
    if child_n["direction"] == "target":
        return [parent_n.id, child_n["node"].id, child_n["rel_type"], 
                child_n['restriction'], child_n['restriction_value'],
                child_n['rel_label'],
               ]
    else:
        return [child_n["node"].id, parent_n.id, child_n["rel_type"], 
                child_n['restriction'], child_n['restriction_value'],
                child_n['rel_label'],
               ]
    

In [12]:
export_traverse(g)

obo.FOODON_00002403
['obo.FOODON_00002403', 'food material', 'http://purl.obolibrary.org/obo/FOODON_00002403', 'Concept']
------------
AND_obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)
['AND_obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)', None, None, 'AND']
['obo.FOODON_00002403', 'AND_obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)', 'equivalent_to', None, None, '']
vvvvvvvvvvvvv
2
obo.ENVO_00010483
['obo.ENVO_00010483', 'environmental material', 'http://purl.obolibrary.org/obo/ENVO_00010483', 'Concept']
['obo.ENVO_00010483', 'AND_obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)', 'member_of', None, None, '']
vvvvvvvvvvvvv
BLANK_obo.RO_0000087.some(obo.CHEBI_33290)
['BLANK_obo.RO_0000087.some(obo.CHEBI_33290)', None, None, 'BLANK']
['BLANK_obo.RO_0000087.some(obo.CHEBI_33290)', 'AND_obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)', 'member_of', None, None, '']
vvvvvvvvvvvvv
1
obo.CHEBI_33290
['obo.CHEBI_33290', 'food', 'http://purl.oboli

In [13]:
type(g)
g.root.children[0]

{'node': <onto_classes.TransitionNode at 0x1157d6ec0>,
 'direction': 'target',
 'rel_type': 'equivalent_to',
 'restriction': None,
 'restriction_value': None}

In [28]:
match g:
    case Graph():
        print(":)")
    case _:
        print(":(")

:)


In [10]:
create_graph(onto)

owl.Thing does not have equivalent_to or is_a properties.


* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_00002511

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_03315876



In [ ]:
'''
Still need to add annotations

Now that graph is successfully created!
Write function to navigate through graph
    Need to check set of nodes before writing to node file!
        Can be harsh with transition nodes and throw an error if not unique?



    start at root and go through all children
    IF type is TRANSITIONNODE WITH NO CHILDREN
        prune
        remove node from transition list
    ELSE
        Check to see if ENTITYNODE is in set
        Write node row csv out
        Write rel row csv out
'''

In [ ]:
raise NotImplementedError

In [8]:
c.equivalent_to[0]

obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)

In [6]:
# create tree for food material
graph = EntityNode(c, "Concept")

# create AND node
and_trans_node = TransitionNode("and_0001", "AND")

# Add AND node to graph
graph.add_child(and_trans_node, direction="source", rel_type="equivalent_to")

# Add links to AND
envo_node = EntityNode(c.equivalent_to[0].is_a[0], "Concept") # obo.ENVO_00010483 -> No children
and_trans_node.add_child(envo_node, direction="source", rel_type="member_of")

# Add BLANK node
blank_trans_node = TransitionNode("BLANK_0001", "BLANK")

# Connect blank node to AND
and_trans_node.add_child(blank_trans_node, direction="target", rel_type="member_of")

# Create entity node and connect with blank node
restriction = c.equivalent_to[0].is_a[1]
chebi_node = EntityNode(restriction.value, "Concept") # obo.ENVO_00010483 -> No children
blank_trans_node.add_child(envo_node, direction="source", rel_type=restriction.property)

In [ ]:
'''
What we really want:
class Graph
    root: starting node
    entity: list
    transition: list
    
functions: export, prune, func to verify nodes not already in list
2 graphs:
    Main graph with complete node list
    Sub graph: each pattern
    
Other classes:
    set_children
        Entity -> check if equivalent_to or is_a is valid list
                  the launch point into creating graph
        Transition -> match on property type (AND, OR, NOT, etc)
                      Unwrap and add nodes + children as appropriate
        
    get_children: retrieve children of node
    
    add_node: Add checks for both entity and transition
        Transition -> Modify __init__ call to take one of those types and create node
    


'''
'''
trace: trace tree and name nodes
    No longer needed since transition nodes can take full name of their description

prune: verify all transition nodes have children
    Def needed to verify integrity of tree due to bad nodes
    
write: write tree out to node and rel csv files
    Still need to add annotations to EntityNodes
'''
# def trace(g):
    

In [12]:
graph.children[0]["node"].children

[{'node': <onto_classes.EntityNode at 0x11f64d4e0>,
  'direction': 'source',
  'rel_type': 'member_of'},
 {'node': <onto_classes.TransitionNode at 0x11f64d360>,
  'direction': 'target',
  'rel_type': 'member_of'}]

In [9]:
c.equivalent_to[0].is_a[1]

obo.RO_0000087.some(obo.CHEBI_33290)

In [10]:
type(c.equivalent_to[0].is_a[1])

owlready2.class_construct.Restriction

In [13]:
"BLANK_" + str(c.equivalent_to[0].is_a[1])

'BLANK_obo.RO_0000087.some(obo.CHEBI_33290)'

In [12]:
c.equivalent_to[0].is_a[1].property

obo.RO_0000087

In [14]:
c.equivalent_to[0].is_a[1].value

obo.CHEBI_33290

In [ ]:
# Add transition node


In [ ]:
# class EntityNode:
#     # def __init__(self, entity, neo_type, parent=None):
#     def __init__(self, entity, neo_type):
#         self.id = str(entity)
#         self.label = ';'.join(entity.label)
#         self.iri = entity.iri
#         self.neo_type = neo_type
#         self.children = []

In [ ]:
# def export_traverse(node):
#     match node:
#         case Graph():
#             # initialize recursion
#             node_children = node.root.children
#             print(node.root.id)
#             print('------------')
            
#             # write out edges and parent node
#             # TODO
#         case EntityNode() | TransitionNode():
#             node_children = node.children
            
#     for n in node_children:
#         print(n['node'].id)
#         print(create_node_row(n['node']))
#         print('vvvvvvvvvvvvv')
#         export_traverse(n['node'])
#         # write out edges
#         #TODO
        
#         # Go through node's children
#         # for child in n["node"].children:
#         #     print(f"\n{child['node'].id}")
#         #     print(create_node_row(child['node']))
#         #     print('+++++++++++++++')
#         #     export_traverse(child['node'])
            
# def create_node_row(n):
#     # node_id:ID,label:string[],iri,:LABEL
#     return [n.id, n.label, n.iri, n.neo_type]